In [5]:
import os
import dotenv
import dsp
dotenv.load_dotenv()


True

In [21]:
openai_api_key = os.environ["OPENAI_API_KEY"]
colbert_server = 'http://ec2-44-228-128-229.us-west-2.compute.amazonaws.com:8893/api/search'

lm = dsp.GPT3(model='text-davinci-002', api_key=openai_api_key)
rm = dsp.ColBERTv2(url=colbert_server)

dsp.settings.configure(lm=lm, rm=rm)

In [7]:
train = [('Who produced the album that included a re-recording of "Lithium"?', ['Butch Vig']),
         ('Who was the director of the 2009 movie featuring Peter Outerbridge as William Easton?', ['Kevin Greutert']),
         ('The heir to the Du Pont family fortune sponsored what wrestling team?', ['Foxcatcher', 'Team Foxcatcher', 'Foxcatcher Team']),
         ('In what year was the star of To Hell and Back born?', ['1925']),
         ('Which award did the first book of Gary Zukav receive?', ['U.S. National Book Award', 'National Book Award']),
         ('What city was the victim of Joseph Druces working in?', ['Boston, Massachusetts', 'Boston']),]

train = [dsp.Example(question=question, answer=answer) for question, answer in train]


In [8]:
dev = [('Who has a broader scope of profession: E. L. Doctorow or Julia Peterkin?', ['E. L. Doctorow', 'E.L. Doctorow', 'Doctorow']),
       ('What documentary about the Gilgo Beach Killer debuted on A&E?', ['The Killing Season']),
       ('Right Back At It Again contains lyrics co-written by the singer born in what city?', ['Gainesville, Florida', 'Gainesville']),
       ('What year was the party of the winner of the 1971 San Francisco mayoral election founded?', ['1828']),
       ('Which author is English: John Braine or Studs Terkel?', ['John Braine']),
       ('Anthony Dirrell is the brother of which super middleweight title holder?', ['Andre Dirrell']),
       ('In which city is the sports nutrition business established by Oliver Cookson based ?', ['Cheshire', 'Cheshire, UK']),
       ('Find the birth date of the actor who played roles in First Wives Club and Searching for the Elephant.', ['February 13, 1980']),
       ('Kyle Moran was born in the town on what river?', ['Castletown', 'Castletown River']),
       ("What is the name of one branch of Robert D. Braun's speciality?", ['aeronautical engineering', 'astronautical engineering', 'aeronautics', 'astronautics']),
       ("Where was the actress who played the niece in the Priest film born?", ['Surrey', 'Guildford, Surrey']),
       ('Name the movie in which the daughter of Noel Harrison plays Violet Trefusis.', ['Portrait of a Marriage']),
       ('What year was the father of the Princes in the Tower born?', ['1442'])]

dev = [dsp.Example(question=question, answer=answer) for question, answer in dev]


In [40]:
Question = dsp.Type(prefix="Question:", desc="${the question to be answered}")
Answer = dsp.Type(prefix="Answer:", desc="${a short factoid answer, often between 1 and 5 words}", format=dsp.format_answers)

qa_template = dsp.Template(instructions="Answer questions with short factoid answers.", question=Question(), answer=Answer())


In [41]:
print(qa_template(dsp.Example(question=dev[0].question, demos=dsp.sample(train, k=1))))

Answer questions with short factoid answers.

---

Follow the following format.

Question: ${the question to be answered}
Answer: ${a short factoid answer, often between 1 and 5 words}

---

Question: Which award did the first book of Gary Zukav receive?
Answer: U.S. National Book Award

Question: Who has a broader scope of profession: E. L. Doctorow or Julia Peterkin?
Answer:


In [46]:
def vanilla_LM_QA(question: str) -> str:
    demos = dsp.sample(train, k=7)
    example = dsp.Example(question=question, demos=demos)

    example, completions = dsp.generate(qa_template)(example, stage='qa')
    return completions.answer


In [50]:
print(dev[0].question)
print(vanilla_LM_QA(dev[0].question))

Who has a broader scope of profession: E. L. Doctorow or Julia Peterkin?
E. L. Doctorow


In [17]:
from dsp.evaluation.utils import evaluate

evaluate(vanilla_LM_QA, dev)


100%|██████████| 13/13 [00:00<00:00, 1802.57it/s]


Answered 3 / 13 (23.1%) correctly.


,question,answer,prediction,correct
0,Who has a broader scope of profession: E. L. Doctorow or Julia Peterkin?,"['E. L. Doctorow', 'E.L. Doctorow', 'Doctorow']",E. L. Doctorow,✔️
1,What documentary about the Gilgo Beach Killer debuted on A&E?,['The Killing Season'],The Long Island Serial Killer,❌
2,Right Back At It Again contains lyrics co-written by the singer born in what city?,"['Gainesville, Florida', 'Gainesville']","Melbourne, Australia",❌
3,What year was the party of the winner of the 1971 San Francisco mayoral election founded?,['1828'],1966,❌
4,Which author is English: John Braine or Studs Terkel?,['John Braine'],John Braine,✔️
5,Anthony Dirrell is the brother of which super middleweight title holder?,['Andre Dirrell'],Andre Dirrell,✔️
6,In which city is the sports nutrition business established by Oliver Cookson based ?,"['Cheshire', 'Cheshire, UK']","Manchester, England",❌
7,Find the birth date of the actor who played roles in First Wives Club and Searching for the Elephant.,"['February 13, 1980']","July 30, 1953",❌
8,Kyle Moran was born in the town on what river?,"['Castletown', 'Castletown River']",Hudson River,❌
9,What is the name of one branch of Robert D. Braun's speciality?,"['aeronautical engineering', 'astronautical engineering', 'aeronautics', 'astronautics']",Aerospace engineering,❌


23.1

In [18]:
Context = dsp.Type(
    prefix="Context:\n",
    desc="${sources that may contain relevant content}",
    format=dsp.passages2text
)

qa_template_with_passages = dsp.Template(
    instructions="Answer questions with short factoid answers.",
    context=Context(), question=Question(), answer=Answer()
)

In [19]:
def retrieve_then_read_QA(question: str) -> str:
    demos = dsp.sample(train, k=7)
    passages = dsp.retrieve(question, k=1)
    
    example = dsp.Example(question=question, context=passages, demos=demos)
    example, completions = dsp.generate(qa_template_with_passages)(example, stage='qa')

    return completions.answer


In [23]:
evaluate(retrieve_then_read_QA, dev)


100%|██████████| 13/13 [00:07<00:00,  1.82it/s]

Answered 5 / 13 (38.5%) correctly.


,question,answer,prediction,correct
0,Who has a broader scope of profession: E. L. Doctorow or Julia Peterkin?,"['E. L. Doctorow', 'E.L. Doctorow', 'Doctorow']",E. L. Doctorow,✔️
1,What documentary about the Gilgo Beach Killer debuted on A&E?,['The Killing Season'],The Killing Season,✔️
2,Right Back At It Again contains lyrics co-written by the singer born in what city?,"['Gainesville, Florida', 'Gainesville']","Daytona Beach, Florida.",❌
3,What year was the party of the winner of the 1971 San Francisco mayoral election founded?,['1828'],1856,❌
4,Which author is English: John Braine or Studs Terkel?,['John Braine'],John Braine is English.,❌
5,Anthony Dirrell is the brother of which super middleweight title holder?,['Andre Dirrell'],Andre Dirrell,✔️
6,In which city is the sports nutrition business established by Oliver Cookson based ?,"['Cheshire', 'Cheshire, UK']","Manchester, England",❌
7,Find the birth date of the actor who played roles in First Wives Club and Searching for the Elephant.,"['February 13, 1980']","August 5, 1962",❌
8,Kyle Moran was born in the town on what river?,"['Castletown', 'Castletown River']","Dundalk, Ireland",❌
9,What is the name of one branch of Robert D. Braun's speciality?,"['aeronautical engineering', 'astronautical engineering', 'aeronautics', 'astronautics']",Jet Propulsion,❌


38.5

In [25]:
Rationale = dsp.Type(
    prefix="Rationale: Let's think step by step.",
    desc="${a step-by-step deduction that identifies the correct response, which will be provided below}"
)

qa_template_with_CoT = dsp.Template(
    instructions=qa_template.instructions,
    context=Context(), question=Question(), rationale=Rationale(), answer=Answer()
)


In [26]:
@dsp.transformation
def QA_predict(example: dsp.Example, sc=True):
    if sc:
        example, completions = dsp.generate(qa_template_with_CoT, n=20, temperature=0.7)(example, stage='qa')
        completions = dsp.majority(completions)
    else:
        example, completions = dsp.generate(qa_template_with_CoT)(example, stage='qa')
    
    return example.copy(answer=completions.answer)

def retrieve_then_read_QA_v2(question: str) -> str:
    demos = dsp.sample(train, k=7)
    passages = dsp.retrieve(question, k=5)
    example = dsp.Example(question=question, context=passages, demos=demos)
    
    return QA_predict(example).answer


In [27]:
retrieve_then_read_QA_v2(dev[2].question), lm.inspect_history(n=1)





Answer questions with short factoid answers.

---

Question: Which award did the first book of Gary Zukav receive?
Answer: U.S. National Book Award

Question: The heir to the Du Pont family fortune sponsored what wrestling team?
Answer: Foxcatcher

Question: Who was the director of the 2009 movie featuring Peter Outerbridge as William Easton?
Answer: Kevin Greutert

Question: Who produced the album that included a re-recording of "Lithium"?
Answer: Butch Vig

Question: What city was the victim of Joseph Druces working in?
Answer: Boston, Massachusetts

Question: In what year was the star of To Hell and Back born?
Answer: 1925

---

Follow the following format.

Context:
${sources that may contain relevant content}

Question: ${the question to be answered}

Rationale: Let's think step by step. ${a step-by-step deduction that identifies the correct response, which will be provided below}

Answer: ${a short factoid answer, often between 1 and 5 words}

---

Context:
[1] «Right Back at

('Ocala, Florida', None)

In [29]:
evaluate(retrieve_then_read_QA_v2, dev)

100%|██████████| 13/13 [02:05<00:00,  9.69s/it]

Answered 7 / 13 (53.8%) correctly.


,question,answer,prediction,correct
0,Who has a broader scope of profession: E. L. Doctorow or Julia Peterkin?,"['E. L. Doctorow', 'E.L. Doctorow', 'Doctorow']",E. L. Doctorow,✔️
1,What documentary about the Gilgo Beach Killer debuted on A&E?,['The Killing Season'],The Killing Season,✔️
2,Right Back At It Again contains lyrics co-written by the singer born in what city?,"['Gainesville, Florida', 'Gainesville']","Ocala, Florida",❌
3,What year was the party of the winner of the 1971 San Francisco mayoral election founded?,['1828'],1824,❌
4,Which author is English: John Braine or Studs Terkel?,['John Braine'],John Braine,✔️
5,Anthony Dirrell is the brother of which super middleweight title holder?,['Andre Dirrell'],Andre Dirrell,✔️
6,In which city is the sports nutrition business established by Oliver Cookson based ?,"['Cheshire', 'Cheshire, UK']","Cheshire, UK",✔️
7,Find the birth date of the actor who played roles in First Wives Club and Searching for the Elephant.,"['February 13, 1980']","September 27, 1974",❌
8,Kyle Moran was born in the town on what river?,"['Castletown', 'Castletown River']",Dundalk,❌
9,What is the name of one branch of Robert D. Braun's speciality?,"['aeronautical engineering', 'astronautical engineering', 'aeronautics', 'astronautics']",aerospace engineering,❌


53.8

In [30]:
SearchRationale = dsp.Type(
    prefix="Rationale: Let's think step by step. To answer this question, we first need to find out",
    desc="${the missing information}"
)

SearchQuery = dsp.Type(
    prefix="Search Query:",
    desc="${a simple question for seeking the missing information}"
)

rewrite_template = dsp.Template(
    instructions="Write a search query that will help answer a complex question.",
    question=Question(), rationale=SearchRationale(), query=SearchQuery()
)

In [31]:
CondenseRationale = dsp.Type(
    prefix="Rationale: Let's think step by step. Based on the context, we have learned the following.",
    desc="${information from the context that provides useful clues}"
)

hop_template = dsp.Template(
    instructions=rewrite_template.instructions,
    context=Context(), question=Question(), rationale=CondenseRationale(), query=SearchQuery()
)


In [32]:
from dsp.utils import deduplicate

@dsp.transformation
def multihop_search_v1(example: dsp.Example, max_hops=2, k=2) -> dsp.Example:
    example.context = []
    
    for hop in range(max_hops):
        # Generate a query based
        template = rewrite_template if hop == 0 else hop_template
        example, completions = dsp.generate(template)(example, stage=f'h{hop}')

        # Retrieve k results based on the query generated
        passages = dsp.retrieve(completions.query, k=k)

        # Update the context by concatenating old and new passages
        example.context = deduplicate(example.context + passages)

    return example


def multihop_QA_v1(question: str) -> str:
    demos = dsp.sample(train, k=7)
    x = dsp.Example(question=question, demos=demos)
    
    x = multihop_search_v1(x)
    x = QA_predict(x, sc=False)

    return x.answer


In [33]:
multihop_QA_v1(dev[2].question), lm.inspect_history(n=3)





Write a search query that will help answer a complex question.

---

Follow the following format.

Question: ${the question to be answered}
Rationale: Let's think step by step. To answer this question, we first need to find out ${the missing information}
Search Query: ${a simple question for seeking the missing information}

---

Question: Right Back At It Again contains lyrics co-written by the singer born in what city?
Rationale: Let's think step by step. To answer this question, we first need to find out the name of the singer
Search Query: Who sings Right Back At It Again







Write a search query that will help answer a complex question.

---

Follow the following format.

Context:
${sources that may contain relevant content}

Question: ${the question to be answered}

Rationale: Let's think step by step. Based on the context, we have learned the following. ${information from the context that provides useful clues}

Search Query: ${a simple question for seeking the missing in

('Gainesville, Florida', None)